In [131]:
from openai import OpenAI

# Point to the local server
def LM_query(query_message):
  client = OpenAI(api_key=INSERT_API_KEY_HERE)#(base_url="http://localhost:1234/v1", api_key="lm-studio")

  completion = client.chat.completions.create(
    model="gpt-4o", #"bartowski/Phi-3-medium-4k-instruct-GGUF",
    messages=PREVIOUS_MESSAGES + [{"role": "user", "content": query_message}],
    temperature=0.7,
    stream = False
  )

  return(completion.choices[0].message.content)

In [139]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

GI_data = pd.read_csv('./GI_data.csv', index_col = 0)

def GI_simple_search(strings, GI_data):
  res = GI_data
  for string in strings:
    res = res[res['Food Name'].str.contains(string, case=False)]
  return res

import re
memo = {} # Assembled dictionary
maybe = [] # Cases needing human suprvision
nope = [] # Not in database
def GI_NL_search(string, GI_data):
  if string in memo:
    return memo[string]

  simplified_string = LM_query('Simplify ' + '\'' + string+ '\'')
  current_strings = []
  for word in simplified_string.split():
    word = word.replace('(', '')
    word = word.replace(')', '')
    word = word.replace('\'', '')
    word = word.replace('\'', '')
    word = word.replace('<', '')
    word = word.replace('>', '')
    word = re.escape(word)
    if word:
      current_strings.append(word)

  while current_strings:
    for idx, row in GI_simple_search(current_strings, GI_data).iterrows():
      ans = LM_query('Accept ' + '\''+ row['Food Name'] + '\''+ ' for ' + '\''+ string + '\''+ '?')[1]
      if ans == 'Y':
        memo[string] = row
        return row
      if ans == 'N':
        break
      if ans == 'M':
        maybe.append((row, string))
        return None
#        if input('Accept ' + '\'' + row['Food Name'] + '\'' + ' for ' + '\'' + string + '\''+ ' ? ') == 'Y':
#          memo[string] = row
#          return row
#        else:
#          return None
    tick = True
    while tick:
      drop = LM_query('Drop ' + '\'' + ' '.join(current_strings) + '\'')[1]
      for no in range(len(current_strings)):
        if str(no) == drop :
          current_strings.pop(no)
          tick = False
          break
  nope.append(string)
  return None

import pickle

with open('all_foods.pkl', 'rb') as f:
    all_foods = pickle.load(f)

all_foods = all_foods - set([''])

First pass: save found foods in 'memo', save unsure comparisons to 'maybe', save foods not found to 'nope'

In [140]:
PREVIOUS_MESSAGES = [
      {
        "role": "system",
        "content": "You are an Assistant with three tasks. Your output should always follow the given form. Your output must not be a sentence.\n\n###\nWhen the prompt is \"Simplify 'String'\" where 'String' is a name of food you will output a simplified, generic name of 'String' which might be found in food databases. Try to use many spaces and simple words.\n\n###\nWhen the prompt is \"Accept 'String1' for 'String2'?\", you will determine whether 'String1' and 'String2' are comparable in terms of Glycemic index and output \"Yes\", \"No\", or \"Maybe\".  The database is sparse, so output 'Yes' if you think the two foods are generally similar in terms of Glycemic index. Sugary drinks are similar to one another, sweet pastries are similar to one another. Only output 'Maybe' if it is really unclear.\n\n###\nWhen the prompt is \"Drop 'Word0 Word1 Word2 ...', output the number which refers to the word which is most irrelevant in terms of Glycemic index."
      },
      {
        "role": "user",
        "content": "Simplify 'Target Brand Extra Firm Tofu, for soups and salads'"
      },
      {
        "role": "assistant",
        "content": " Extra Firm Tofu"
      },
      {
        "role": "user",
        "content": "Accept 'Kroger Choice Extra firm tofu' for 'Silken Tofu'?"
      },
      {
        "role": "assistant",
        "content": " Yes"
      },
      {
        "role": "user",
        "content": "Drop 'Beef Jerky'"
      },
      {
        "role": "assistant",
        "content": " 0"
      },
      {
        "role": "user",
        "content": "Simplify 'Half and Half'"
      },
      {
        "role": "assistant",
        "content": " Half fat cream"
      },
      {
        "role": "user",
        "content": "Accept 'Ribeye Streak, USDA Prime' for 'Ribeye Streak, USDA Choice'?"
      },
      {
        "role": "assistant",
        "content": " Yes"
      },
      {
        "role": "user",
        "content": "Accept 'Baked beans, in BBQ sauce' for 'Baked beans'?"
      },
      {
        "role": "assistant",
        "content": " Yes"
      },{
        "role": "user",
        "content": "Accept 'Baked beans, in BBQ sauce' for 'Black beans'?"
      },
      {
        "role": "assistant",
        "content": " Maybe"
      },
      {
        "role": "user",
        "content": "Drop 'Chocolate filled Cake'"
      },
      {
        "role": "assistant",
        "content": " 1"
      },
      {
        "role": "user",
        "content": "Simplify 'Chocolate Mudcake'"
      },
      {
        "role": "assistant",
        "content": " Chocolate Cake"
      },
      {
        "role": "user",
        "content": "Accept 'Ribeye Streak, USDA Prime' for 'Tenderloin, USDA Prime'?"
      },
      {
        "role": "assistant",
        "content": " No"
      }
    ]

  

In [141]:
food_no = 0
for food in list(all_foods):
  if GI_NL_search(food, GI_data) is not None:
    food_no += 1
    print(str(food_no) + ' added')
    with open('memo.pkl', 'wb') as f:
      pickle.dump(memo, f)
  else:
    food_no += 1
    print(food_no, 'not added. ', 'Maybe: ', len(maybe), ', Nope: ', len(nope))

1 not added.  Maybe:  0 , Nope:  1
2 added
3 not added.  Maybe:  0 , Nope:  2
4 not added.  Maybe:  0 , Nope:  3
5 not added.  Maybe:  0 , Nope:  4
6 not added.  Maybe:  1 , Nope:  4
7 not added.  Maybe:  2 , Nope:  4
8 not added.  Maybe:  2 , Nope:  5
9 not added.  Maybe:  2 , Nope:  6
10 not added.  Maybe:  2 , Nope:  7
11 not added.  Maybe:  2 , Nope:  8
12 not added.  Maybe:  2 , Nope:  9
13 not added.  Maybe:  2 , Nope:  10
14 not added.  Maybe:  3 , Nope:  10
15 not added.  Maybe:  3 , Nope:  11
16 not added.  Maybe:  3 , Nope:  12
17 not added.  Maybe:  4 , Nope:  12
18 not added.  Maybe:  5 , Nope:  12
19 not added.  Maybe:  5 , Nope:  13
20 not added.  Maybe:  6 , Nope:  13
21 not added.  Maybe:  6 , Nope:  14
22 not added.  Maybe:  6 , Nope:  15
23 not added.  Maybe:  6 , Nope:  16
24 not added.  Maybe:  6 , Nope:  17
25 not added.  Maybe:  6 , Nope:  18
26 not added.  Maybe:  6 , Nope:  19
27 not added.  Maybe:  6 , Nope:  20
28 not added.  Maybe:  6 , Nope:  21
29 added
30 

Repeat look at maybes, which ended up sending everything to nope

In [152]:
PREVIOUS_MESSAGES = [
      {
        "role": "system",
        "content": "You are an Assistant classifying whether two given foods are equivalent in terms of Glycemic index. If prompted When the prompt is \"Accept 'String1' for 'String2'?\", you will determine whether 'String1' and 'String2' are comparable in terms of Glycemic index and output \"Yes\" if they are equivalent or \"No\" if they aren't or if you are not sure."
      }]

In [158]:
new_maybe = []
maybe_no = 0
for (row, string) in maybe:
    ans = LM_query('Accept ' + '\''+ row['Food Name'] + '\''+ ' for ' + '\''+ string + '\''+ '?')[0]
    if ans == 'Y':
        memo[string] = row
    else:
        if ans == 'N':
            nope.append(string)
            print('Nope added')
    maybe_no += 1
    print('Entry', maybe_no)

Nope added
Entry 1
Nope added
Entry 2
Nope added
Entry 3
Nope added
Entry 4
Nope added
Entry 5
Nope added
Entry 6
Nope added
Entry 7
Nope added
Entry 8
Nope added
Entry 9
Nope added
Entry 10
Nope added
Entry 11
Nope added
Entry 12
Nope added
Entry 13
Nope added
Entry 14
Nope added
Entry 15
Nope added
Entry 16
Nope added
Entry 17
Nope added
Entry 18
Nope added
Entry 19
Nope added
Entry 20
Nope added
Entry 21
Nope added
Entry 22
Nope added
Entry 23
Nope added
Entry 24
Nope added
Entry 25
Nope added
Entry 26
Nope added
Entry 27
Nope added
Entry 28
Nope added
Entry 29
Nope added
Entry 30
Nope added
Entry 31
Nope added
Entry 32
Nope added
Entry 33
Nope added
Entry 34
Nope added
Entry 35
Nope added
Entry 36
Nope added
Entry 37
Nope added
Entry 38
Nope added
Entry 39
Nope added
Entry 40
Nope added
Entry 41
Nope added
Entry 42
Nope added
Entry 43
Nope added
Entry 44
Nope added
Entry 45
Nope added
Entry 46
Nope added
Entry 47
Nope added
Entry 48
Nope added
Entry 49
Nope added
Entry 50
Nope adde

High-Low-Med guesses for nopes

In [159]:
PREVIOUS_MESSAGES = [
      {
        "role": "system",
        "content": "You are an Assistant classifying the food item given in the prompt into High, Med, or Low Glycemic Index (GI). Do your best to classify, knowing that carbohydrates raise GI. Fats tend to lower GI slightly. Your output should be one of 'High', 'Low', or 'Med', nothing else."
      }]

In [163]:
GI_guesses = {}
for string in nope:
    print(len(GI_guesses))
    while True:
        ans = LM_query(string)
        if ans not in ['High', 'Low', 'Med']:
            continue
        else:
            GI_guesses[string] = ans
            break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [166]:
with open('GI_known_dict.pkl', 'wb') as f:
    pickle.dump(memo, f)
with open('GI_guessed_dict.pkl', 'wb') as f:
    pickle.dump(GI_guesses, f)